# TODO
-

# 1. Settings

## 1.1 User variables

In [1]:
MIN_CHARCOAL = 0.1

In [2]:
NB_RUNS = 10

USE_DISCRETE = False

#TARGET = "charcoal_change"

NB_TOP_MODELS = 10
TRAIN_SIZE = 0.8
FOLD_STRATEGY = "kfold"
N_FOLD = 5
FIX_IMBALANCE = True # Good for decision tree
FEATURE_SELECTION = False
N_FEATURES_TO_SELECT = 0.2
REMOVE_MULTICOLLINEARITY = False # Good for ada boost sometime
PREPROCESS = True
TRANSFORMATION = True
TRANSFORMATION_METHOD = 'yeo-johnson'
NORMALIZE = True
NORMALIZE_METHOD = "zscore" #maxabs

PCA = False
POLYNOMIAL_FEATURES = False

SORT_METRIC = "MAE"
OPTIMIZE_METRIC = "MAE"

EXCLUDED_MODELS = ["lar"]#["catboost", "nb", "dummy", "lr", "qda", "knn", "lda", "svm", "ridge"] # ["catboost", "Naive Bayes", "Dummy Classifier", "Logistic Regression", "Quadratic Discriminant Analysis", "K Neighbors Classifier"]

# 1.2 Dev variables

In [3]:
RANDOM_SEED = 0

INPUT_DATA = "tmp/data_processed.csv"
INPUT_DATA_DISCRETE = "tmp/data_discrete.csv"

OUTPUT_SCORES_PATH = "tmp/hap_pycaret_scores.csv"
VERBOSE = True

## 1.3 Imports

In [4]:
import pandas as pd
import numpy as np
import random
from pycaret.regression import *

from config import *

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

## 1.4 Constants

In [5]:
# Constants
DEBUG = False

# 2. Data Loading

In [6]:
if USE_DISCRETE:
    df = pd.read_csv(INPUT_DATA_DISCRETE)
else:
    df = pd.read_csv(INPUT_DATA)

#df = df[df[TARGET] >= MIN_CHARCOAL]
display(df)

,Abies,Juniperus,Larix,Picea,Pinus,Tsuga,Acer,Alnus,Betula,Carpinus,...,Phytolithe,Poaceae,Rosaceae,Sarcobatus,Thalictrum,Typha,Spore_monolete_,Spore_trilete,Aquatics,Abies_prev
0,0.189546,0.302115,0.604230,13.897281,62.235650,0.302115,0.000000,0.604230,10.574018,0.000000,...,0.000000,1.208459,0.000000,0.000000,0.000000,0.000000,0.60423,0.302115,0.000000,1.095890
1,0.066008,0.840336,1.680672,14.005602,59.383753,0.000000,0.000000,0.560224,9.523810,0.000000,...,0.000000,1.400560,0.000000,0.000000,0.000000,0.280112,0.00000,0.000000,0.000000,0.906344
2,-0.216002,0.264085,2.640845,12.147887,52.816901,0.000000,0.000000,5.545775,13.204225,0.264085,...,0.000000,1.584507,0.264085,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.840336
3,-0.687848,0.000000,1.744186,16.694352,55.315615,0.000000,0.249169,0.996678,9.219269,0.000000,...,0.000000,4.235880,0.000000,0.000000,0.249169,0.000000,0.00000,0.000000,0.000000,1.056338
4,0.077519,0.000000,0.833333,10.000000,56.111111,0.000000,0.000000,3.888889,13.333333,0.000000,...,0.277778,1.944444,0.277778,0.000000,0.277778,0.000000,0.00000,0.000000,0.000000,1.744186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,0.000000,0.307062,2.456499,25.793245,31.320368,0.000000,0.000000,0.307062,21.494371,0.000000,...,0.000000,3.991812,0.000000,0.000000,0.000000,1.535312,0.00000,0.000000,0.000000,0.000000
57,0.000000,0.568182,1.136364,13.068182,51.704545,0.000000,0.000000,0.000000,15.909091,0.000000,...,0.000000,5.681818,0.000000,0.284091,0.000000,0.284091,0.00000,0.000000,0.284091,0.000000
58,0.000000,1.863354,0.621118,10.559006,57.142857,0.000000,0.000000,0.310559,12.422360,0.000000,...,0.000000,3.416149,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
59,0.000000,0.595238,1.785714,17.559524,54.166667,0.000000,0.000000,0.000000,10.119048,0.000000,...,0.000000,3.273810,0.000000,0.000000,0.000000,0.297619,0.00000,0.000000,0.297619,0.000000


# Pycaret AutoML

In [7]:
df_scores = None

setup(df, target = TARGET, session_id = RANDOM_SEED,
    train_size=TRAIN_SIZE,
    fold_strategy=FOLD_STRATEGY,
    fold=N_FOLD,
    #fix_imbalance=FIX_IMBALANCE,
    preprocess=PREPROCESS,
    feature_selection=FEATURE_SELECTION,
    n_features_to_select=N_FEATURES_TO_SELECT,
    remove_multicollinearity=REMOVE_MULTICOLLINEARITY,
    normalize=NORMALIZE,
    normalize_method=NORMALIZE_METHOD,
    pca=PCA,
    polynomial_features=POLYNOMIAL_FEATURES,
    index=False,
    n_jobs=-1
    )

print("Val Scores")
top = compare_models(n_select = NB_TOP_MODELS,
                    sort=SORT_METRIC,
                    exclude=EXCLUDED_MODELS,
                    verbose=VERBOSE,
                    errors="raise")
df_results = pull()
df_results

,Description,Value
0,Session id,0
1,Target,Abies
2,Target type,Regression
3,Original data shape,"(61, 41)"
4,Transformed data shape,"(61, 41)"
5,Transformed train set shape,"(48, 41)"
6,Transformed test set shape,"(13, 41)"
7,Numeric features,40
8,Preprocess,True
9,Imputation type,simple


Val Scores


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
ada,AdaBoost Regressor,0.2419,0.2130,0.3994,-0.2214,0.1825,0.8267,0.0140
rf,Random Forest Regressor,0.2761,0.2467,0.4551,-0.3427,0.1885,1.2642,0.0320
et,Extra Trees Regressor,0.2774,0.2424,0.4486,-0.2661,0.1933,0.8764,0.0200
dt,Decision Tree Regressor,0.2825,0.3137,0.4987,-0.9222,0.2387,1.8380,0.0060
br,Bayesian Ridge,0.2942,0.2650,0.5021,-0.3789,0.2999,1.0102,0.0120
lasso,Lasso Regression,0.2946,0.2649,0.5021,-0.3780,0.3010,1.0109,0.2420
en,Elastic Net,0.2946,0.2649,0.5021,-0.3780,0.3010,1.0109,0.0060
llar,Lasso Least Angle Regression,0.2946,0.2649,0.5021,-0.3780,0.3010,1.0109,0.0060
lightgbm,Light Gradient Boosting Machine,0.2946,0.2649,0.5021,-0.3780,0.3010,1.0109,0.0080
dummy,Dummy Regressor,0.2946,0.2649,0.5021,-0.3780,0.3010,1.0109,0.0060


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
ada,AdaBoost Regressor,0.2419,0.2130,0.3994,-0.2214,0.1825,0.8267,0.014
rf,Random Forest Regressor,0.2761,0.2467,0.4551,-0.3427,0.1885,1.2642,0.032
et,Extra Trees Regressor,0.2774,0.2424,0.4486,-0.2661,0.1933,0.8764,0.020
dt,Decision Tree Regressor,0.2825,0.3137,0.4987,-0.9222,0.2387,1.8380,0.006
br,Bayesian Ridge,0.2942,0.2650,0.5021,-0.3789,0.2999,1.0102,0.012
lasso,Lasso Regression,0.2946,0.2649,0.5021,-0.3780,0.3010,1.0109,0.242
en,Elastic Net,0.2946,0.2649,0.5021,-0.3780,0.3010,1.0109,0.006
llar,Lasso Least Angle Regression,0.2946,0.2649,0.5021,-0.3780,0.3010,1.0109,0.006
lightgbm,Light Gradient Boosting Machine,0.2946,0.2649,0.5021,-0.3780,0.3010,1.0109,0.008
dummy,Dummy Regressor,0.2946,0.2649,0.5021,-0.3780,0.3010,1.0109,0.006


# DEBUG

In [8]:
evaluate_model(top)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

# TODO